In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import torch
from efficientnet import EfficientNet
from tqdm.notebook import tqdm
import pickle
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid, save_image


In [20]:
# networKIN data를 불러온다.
df_nwk = pd.read_csv('/home/hb/python/pp_pred/raw_data/220531_GPS5.0-NetworKIN/curated.networkin_0111 (1).txt', sep="\t")
display(df_nwk.head(2))
print(df_nwk.columns)

,#Name,Position,Tree,NetPhorest Group,Kinase/Phosphatase/Phospho-binding domain,NetworKIN score,NetPhorest probability,STRING score,Target STRING ID,Kinase/Phosphatase/Phospho-binding domain STRING ID,Target description,Kinase/Phosphatase/Phospho-binding domain description,Target Name,Kinase/Phosphatase/Phospho-binding domain Name,Peptide sequence window,Intermediate nodes
0,Q9GZM8,S242,WW,any_group,PIN1,1.1188,0.1258,0.3862,ENSP00000333982,ENSP00000247970,NDEL1,PIN1,nudE nuclear distribution gene E homolog (A. n...,"peptidylprolyl cis/trans isomerase, NIMA-inter...",NGFGTsPLTPS,ENSP00000300093
1,Q9GZM8,S242,KIN,CDK2_CDK3_CDK1_CDK5_group,CDK1,6.3427,0.1481,0.7380,ENSP00000333982,ENSP00000306043,NDEL1,CDK1,nudE nuclear distribution gene E homolog (A. n...,cyclin-dependent kinase 1,NGFGTsPLTPS,NaN


Index(['#Name', 'Position', 'Tree', 'NetPhorest Group',
       'Kinase/Phosphatase/Phospho-binding domain', 'NetworKIN score',
       'NetPhorest probability', 'STRING score', 'Target STRING ID',
       'Kinase/Phosphatase/Phospho-binding domain STRING ID',
       'Target description',
       'Kinase/Phosphatase/Phospho-binding domain description', 'Target Name',
       'Kinase/Phosphatase/Phospho-binding domain Name',
       'Peptide sequence window', 'Intermediate nodes'],
      dtype='object')


In [21]:
# gps data를 불러온다.
df_gps = pd.read_csv('/home/hb/python/pp_pred/raw_data/220531_GPS5.0-NetworKIN/networkinsub.gps.csv', sep='\t')
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386


In [22]:
# gps data의 Peptide길이는 15인데, networKIN data의 peptide 길이는 11이다.
# 따라서 gps data의 peptide를 변경하여 길이를 11로 만들어주는 작업을 한다.
df_gps[['peptide']] = 0
df_gps['peptide'] = df_gps['Peptide'].apply(lambda x:x[2:13])
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff,peptide
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386,TYYKASKKFRT
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386,KFRTPSFLKKS
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386,DASRPSSSRSY


In [23]:
df_gps['gps_kin_sub_seq'] = df_gps['Kinase'].str.cat(df_gps['peptide'], sep='_')
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff,peptide,gps_kin_sub_seq
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386,TYYKASKKFRT,AGC_TYYKASKKFRT
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386,KFRTPSFLKKS,AGC_KFRTPSFLKKS
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386,DASRPSSSRSY,AGC_DASRPSSSRSY


In [30]:
gps_kin_sub_seq = df_gps[['gps_kin_sub_seq']]
gps_kin_sub_seq['new'] = gps_kin_sub_seq['gps_kin_sub_seq'].apply(lambda x:x.split('/'))
gps_kin_sub_seq['new'] = gps_kin_sub_seq['new'].apply(lambda x:x[-1])
gps_kin_sub_seq = gps_kin_sub_seq[['new']]
gps_kin_sub_seq

/tmp/ipykernel_2281604/75081345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_kin_sub_seq['new'] = gps_kin_sub_seq['gps_kin_sub_seq'].apply(lambda x:x.split('/'))
/tmp/ipykernel_2281604/75081345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_kin_sub_seq['new'] = gps_kin_sub_seq['new'].apply(lambda x:x[-1])


,new
0,AGC_TYYKASKKFRT
1,AGC_KFRTPSFLKKS
2,AGC_DASRPSSSRSY
3,AGC_ASRPSSSRSYV
4,AGC_LEKRASGQAFE
...,...
34778,WEE1_NLIREYVKQTW
34779,WEE1_GMMSDYGTARR
34780,WEE1_LVASLYASSPG
34781,WEE1_ENFDDYMKEVG


In [32]:
df_nwk.head(3)

,#Name,Position,Tree,NetPhorest Group,Kinase/Phosphatase/Phospho-binding domain,NetworKIN score,NetPhorest probability,STRING score,Target STRING ID,Kinase/Phosphatase/Phospho-binding domain STRING ID,Target description,Kinase/Phosphatase/Phospho-binding domain description,Target Name,Kinase/Phosphatase/Phospho-binding domain Name,Peptide sequence window,Intermediate nodes
0,Q9GZM8,S242,WW,any_group,PIN1,1.1188,0.1258,0.3862,ENSP00000333982,ENSP00000247970,NDEL1,PIN1,nudE nuclear distribution gene E homolog (A. n...,"peptidylprolyl cis/trans isomerase, NIMA-inter...",NGFGTsPLTPS,ENSP00000300093
1,Q9GZM8,S242,KIN,CDK2_CDK3_CDK1_CDK5_group,CDK1,6.3427,0.1481,0.7380,ENSP00000333982,ENSP00000306043,NDEL1,CDK1,nudE nuclear distribution gene E homolog (A. n...,cyclin-dependent kinase 1,NGFGTsPLTPS,NaN
2,Q9GZM8,S242,KIN,AuroraA,AuroraA,5.3682,0.0093,0.9410,ENSP00000333982,ENSP00000216911,NDEL1,AURKA,nudE nuclear distribution gene E homolog (A. n...,aurora kinase A,NGFGTsPLTPS,NaN


In [33]:
df_nwk['nwk_kin_sub_seq'] = df_nwk['Kinase/Phosphatase/Phospho-binding domain description'].str.cat(df_nwk['Peptide sequence window'], sep='_')
df_nwk.head(3)

,#Name,Position,Tree,NetPhorest Group,Kinase/Phosphatase/Phospho-binding domain,NetworKIN score,NetPhorest probability,STRING score,Target STRING ID,Kinase/Phosphatase/Phospho-binding domain STRING ID,Target description,Kinase/Phosphatase/Phospho-binding domain description,Target Name,Kinase/Phosphatase/Phospho-binding domain Name,Peptide sequence window,Intermediate nodes,nwk_kin_sub_seq
0,Q9GZM8,S242,WW,any_group,PIN1,1.1188,0.1258,0.3862,ENSP00000333982,ENSP00000247970,NDEL1,PIN1,nudE nuclear distribution gene E homolog (A. n...,"peptidylprolyl cis/trans isomerase, NIMA-inter...",NGFGTsPLTPS,ENSP00000300093,PIN1_NGFGTsPLTPS
1,Q9GZM8,S242,KIN,CDK2_CDK3_CDK1_CDK5_group,CDK1,6.3427,0.1481,0.7380,ENSP00000333982,ENSP00000306043,NDEL1,CDK1,nudE nuclear distribution gene E homolog (A. n...,cyclin-dependent kinase 1,NGFGTsPLTPS,NaN,CDK1_NGFGTsPLTPS
2,Q9GZM8,S242,KIN,AuroraA,AuroraA,5.3682,0.0093,0.9410,ENSP00000333982,ENSP00000216911,NDEL1,AURKA,nudE nuclear distribution gene E homolog (A. n...,aurora kinase A,NGFGTsPLTPS,NaN,AURKA_NGFGTsPLTPS


In [35]:
nwk_kin_sub_seq = df_nwk[['nwk_kin_sub_seq']]
nwk_kin_sub_seq.head(3)

nwk_kin_sub_seq['nwk_kin_sub_seq'] = nwk_kin_sub_seq['nwk_kin_sub_seq'].apply(lambda x:x.upper())
nwk_kin_sub_seq


/tmp/ipykernel_2281604/4140347036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nwk_kin_sub_seq['nwk_kin_sub_seq'] = nwk_kin_sub_seq['nwk_kin_sub_seq'].apply(lambda x:x.upper())


,nwk_kin_sub_seq
0,PIN1_NGFGTSPLTPS
1,CDK1_NGFGTSPLTPS
2,AURKA_NGFGTSPLTPS
3,NEK2_NGFGTSPLTPS
4,CDK5_NGFGTSPLTPS
...,...
42574,YWHAQ_AESGKSASTEV
42575,YWHAG_AESGKSASTEV
42576,YWHAB_AESGKSASTEV
42577,YWHAH_AESGKSASTEV


In [43]:
gps_kin_sub_seq

,kin_subseq
0,AGC_TYYKASKKFRT
1,AGC_KFRTPSFLKKS
2,AGC_DASRPSSSRSY
3,AGC_ASRPSSSRSYV
4,AGC_LEKRASGQAFE
...,...
34756,WEE1B_GQESEYGNITY
34757,WEE1B_GGQKPYPSPKT
34758,WEE1B_GFLQRYSSDPT
34759,WEE1B_EEDETYGVVYK


In [41]:
print(len(nwk_kin_sub_seq))
nwk_kin_sub_seq.drop_duplicates(inplace=True)
len(nwk_kin_sub_seq)

print(len(gps_kin_sub_seq))
gps_kin_sub_seq.drop_duplicates(inplace=True)
len(gps_kin_sub_seq)

42579


/tmp/ipykernel_2281604/4090322876.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nwk_kin_sub_seq.drop_duplicates(inplace=True)


40031

In [37]:
nwk_kin_sub_seq.columns = ['kin_subseq']
display(nwk_kin_sub_seq)

gps_kin_sub_seq.columns = ['kin_subseq']
gps_kin_sub_seq

,kin_subseq
0,PIN1_NGFGTSPLTPS
1,CDK1_NGFGTSPLTPS
2,AURKA_NGFGTSPLTPS
3,NEK2_NGFGTSPLTPS
4,CDK5_NGFGTSPLTPS
...,...
42574,YWHAQ_AESGKSASTEV
42575,YWHAG_AESGKSASTEV
42576,YWHAB_AESGKSASTEV
42577,YWHAH_AESGKSASTEV


,kin_subseq
0,AGC_TYYKASKKFRT
1,AGC_KFRTPSFLKKS
2,AGC_DASRPSSSRSY
3,AGC_ASRPSSSRSYV
4,AGC_LEKRASGQAFE
...,...
34778,WEE1_NLIREYVKQTW
34779,WEE1_GMMSDYGTARR
34780,WEE1_LVASLYASSPG
34781,WEE1_ENFDDYMKEVG


In [44]:
gps_nwk_merge = pd.merge(gps_kin_sub_seq, nwk_kin_sub_seq, on = "kin_subseq", how='inner')
display(gps_nwk_merge.head(3))
len(gps_nwk_merge)

,kin_subseq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE


4135

In [54]:
  gps_nwk_merge.to_csv("/home/hb/python/pp_pred/raw_data/220601_pp-pred_(gps_networKIN_중복데이터비교)-v1.0_HB.csv")



In [46]:


gps_nwk_seq = pd.read_csv('/home/hb/python/pp_pred/raw_data/220610_pp-pred_(gps_networKIN_중복데이터비교_from_KW)-v1.0_HB.csv')


In [51]:
gps_nwk_merge

,kin_subseq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE
3,DMPK_LSRRPSYRKIL
4,DMPK_RMRRNSFTPLS
...,...
4130,MAP2K7_EDIKSYYTVRQ
4131,MAP2K7_EDSTYYKASKG
4132,MAP2K7_PKGTGYIKTEL
4133,MAP2K7_CPQAHYNMYPQ


In [57]:
gps_nwk_merge.columns = ['kin_sub_seq']
gps_nwk_merge

,kin_sub_seq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE
3,DMPK_LSRRPSYRKIL
4,DMPK_RMRRNSFTPLS
...,...
4130,MAP2K7_EDIKSYYTVRQ
4131,MAP2K7_EDSTYYKASKG
4132,MAP2K7_PKGTGYIKTEL
4133,MAP2K7_CPQAHYNMYPQ


In [48]:
gps_nwk_seq = gps_nwk_seq[['kin_sub_seq']]

In [49]:
# GPSanswer[pd.concat([networkin,GPSanswer]).duplicated(subset=['Kinase','Peptide'],keep=False)[40031:]]

gps_nwk_seq

,kin_sub_seq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE
3,DMPK_LSRRPSYRKIL
4,DMPK_RMRRNSFTPLS
...,...
4185,MAP2K7_EDIKSYYTVRQ
4186,MAP2K7_EDSTYYKASKG
4187,MAP2K7_PKGTGYIKTEL
4188,MAP2K7_CPQAHYNMYPQ


In [58]:
seq = pd.concat([gps_nwk_merge, gps_nwk_seq],axis=0)

In [59]:
seq

,kin_sub_seq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE
3,DMPK_LSRRPSYRKIL
4,DMPK_RMRRNSFTPLS
...,...
4185,MAP2K7_EDIKSYYTVRQ
4186,MAP2K7_EDSTYYKASKG
4187,MAP2K7_PKGTGYIKTEL
4188,MAP2K7_CPQAHYNMYPQ


In [62]:
seq[~(seq.duplicated())]

,kin_sub_seq
0,DMPK_KFRTPSFLKKS
1,DMPK_LEKRASGQAFE
2,DMPK_LVRSASSDTSE
3,DMPK_LSRRPSYRKIL
4,DMPK_RMRRNSFTPLS
...,...
375,TEC_FEGFSYVNPQF
376,TEC_EEEHVYSFPNK
377,TEC_EDIKSYYTVRQ
382,TEC_SEETPYSYPTG


In [82]:
seq2

,kin_sub_seq
130,CSK_ETKSLYPSSEI
138,CSK_DKQVEYLDLDL
151,CSK_KTDRIYAMKVV
157,CSK_EDIKSYYTVRQ
199,FER_IESDIYAEIPD
200,FER_DGKEIYNTIRR
201,FER_TGNHTYQEIAV
203,FER_DKQVEYLDLDL
205,FER_KKPTMYPEWKS
206,FER_IIEDPYYGNDS


In [65]:
seq2 = seq.drop_duplicates(['kin_sub_seq'], keep = False)
len(seq2)

55

In [69]:
'AGC_TYYKASKKFRT'.find('df_gps[[g')

-1

In [75]:
gpsseq = list(df_gps['gps_kin_sub_seq'])
type(gpsseq)

list

In [73]:
gpsseq[0]

'gps_kin_sub_seq'

In [81]:
nwkseq.index('CSK_DKQVEYLDLDL')

ValueError: 'CSK_DKQVEYLDLDL' is not in list

In [80]:
nwkseq = list(df_nwk['nwk_kin_sub_seq'])
nwkseq

['PIN1_NGFGTsPLTPS',
 'CDK1_NGFGTsPLTPS',
 'AURKA_NGFGTsPLTPS',
 'NEK2_NGFGTsPLTPS',
 'CDK5_NGFGTsPLTPS',
 'MAPK12_NGFGTsPLTPS',
 'MAPK3_NGFGTsPLTPS',
 'MAPK11_NGFGTsPLTPS',
 'CDK2_NGFGTsPLTPS',
 'MAPK10_NGFGTsPLTPS',
 'MAPK13_NGFGTsPLTPS',
 'MAPK1_NGFGTsPLTPS',
 'CDK3_NGFGTsPLTPS',
 'PRKCZ_NGFGTsPLTPS',
 'MAPK8_NGFGTsPLTPS',
 'CDK6_NGFGTsPLTPS',
 'RPS6KA1_NGFGTsPLTPS',
 'PRKG1_NGFGTsPLTPS',
 'NLK_NGFGTsPLTPS',
 'CDC42BPA_NGFGTsPLTPS',
 'CSNK1E_NGFGTsPLTPS',
 'CLK2_NGFGTsPLTPS',
 'CSNK1G2_NGFGTsPLTPS',
 'CSNK1G1_NGFGTsPLTPS',
 'CSNK1G3_NGFGTsPLTPS',
 'NEK11_NGFGTsPLTPS',
 'ROCK2_NGFGTsPLTPS',
 'NEK4_NGFGTsPLTPS',
 'RPS6KA3_NGFGTsPLTPS',
 'NEK1_NGFGTsPLTPS',
 'CLK3_NGFGTsPLTPS',
 'NEK3_NGFGTsPLTPS',
 'MAK_NGFGTsPLTPS',
 'CDC42BPG_NGFGTsPLTPS',
 'MOK_NGFGTsPLTPS',
 'ICK_NGFGTsPLTPS',
 'CLK4_NGFGTsPLTPS',
 'CLK1_NGFGTsPLTPS',
 'CDC42BPB_NGFGTsPLTPS',
 'DMPK_NGFGTsPLTPS',
 'CAMK1_NGFGTsPLTPS',
 'CAMK1G_NGFGTsPLTPS',
 'PNCK_NGFGTsPLTPS',
 'CAMK1D_NGFGTsPLTPS',
 'PRKCD_NGFGTsPLTPS',
 'PDK4_N

In [67]:
df_gps

,Position,Code,Kinase,Peptide,Score,Cutoff,peptide,gps_kin_sub_seq
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386,TYYKASKKFRT,AGC_TYYKASKKFRT
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386,KFRTPSFLKKS,AGC_KFRTPSFLKKS
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386,DASRPSSSRSY,AGC_DASRPSSSRSY
3,123,S,AGC,PDASRPSSSRSYVLE,1.638,0.386,ASRPSSSRSYV,AGC_ASRPSSSRSYV
4,134,S,AGC,YVLEKRASGQAFEEA,1.645,0.386,LEKRASGQAFE,AGC_LEKRASGQAFE
...,...,...,...,...,...,...,...,...
34778,2575,Y,Dual/Other/WEE/WEE1/WEE1,SKNLIREYVKQTWPK,1.775,1.668,NLIREYVKQTW,Dual/Other/WEE/WEE1/WEE1_NLIREYVKQTW
34779,2749,Y,Dual/Other/WEE/WEE1/WEE1,QAGMMSDYGTARRNA,2.518,1.668,GMMSDYGTARR,Dual/Other/WEE/WEE1/WEE1_GMMSDYGTARR
34780,2859,Y,Dual/Other/WEE/WEE1/WEE1,EELVASLYASSPGGV,3.013,1.668,LVASLYASSPG,Dual/Other/WEE/WEE1/WEE1_LVASLYASSPG
34781,2914,Y,Dual/Other/WEE/WEE1/WEE1,PAENFDDYMKEVGQA,2.546,1.668,ENFDDYMKEVG,Dual/Other/WEE/WEE1/WEE1_ENFDDYMKEVG


In [66]:
seq2 # 건원님 중복 리스트에는 있고, 제 중복 리스트에는 없는 값들 -> 55개

,kin_sub_seq
130,CSK_ETKSLYPSSEI
138,CSK_DKQVEYLDLDL
151,CSK_KTDRIYAMKVV
157,CSK_EDIKSYYTVRQ
199,FER_IESDIYAEIPD
200,FER_DGKEIYNTIRR
201,FER_TGNHTYQEIAV
203,FER_DKQVEYLDLDL
205,FER_KKPTMYPEWKS
206,FER_IIEDPYYGNDS


In [83]:
  seq2.to_csv("/home/hb/python/pp_pred/result/중복데이터55개.csv")
